<a href="https://colab.research.google.com/github/elbyvaz/data_engineering/blob/main/spark/random_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# veritying python version
# !python --version

Python 3.10.12


In [ ]:
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=4d250811b4744f22fab7d531317f3807386b047912aecef86b86de21aaea753b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# using spark and veritying its version
from pyspark.sql import SparkSession

spark = SparkSession \
                    .builder \
                    .appName('Eng. dados - Tecnologia de Big Data - aula 2.2') \
                    .getOrCreate()

spark.version

'3.5.0'

In [ ]:
filename = "/content/sample_data/numbers.txt" # precisa referenciar a estrutura de diretorios do google colab, q eh linux
text_df = spark.read.text(filename)
# Apply a transformation to extract the first element of each row
linesRdd = text_df.rdd.map(lambda r: r[0]) # rdd: estrutura de dado do python q abstrai o cluster, como se estivesse na maq local
print('Number of partitions: {}'.format(linesRdd.getNumPartitions()))

# Collect the results (for demonstration purposes)
# result = linesRdd.collect()

# Display the results
# print(result)

Number of partitions: 2


In [ ]:
"""
The objective of code below is to count how many times each number was showed in the file
Sample file:
  1 9 8 10 1 9 2
Result:
  1 -> 2  # 2 times
  9 -> 2
  8 -> 1
  10 -> 1
  2 -> 1
"""
from operator import add

"""
linesRdd.sample(False, 0.01): only 1% of the lines for test
flatMap: ?
filter: only pair numbers
map & reduce: responsible for counting; add 1 to every equal number
"""
countsRdd = linesRdd.sample(False, 0.01) \
                    .flatMap(lambda line: line.split(' ')) \
                    .filter(lambda number: int(number)%2==0) \
                    .map(lambda number: (number, 1)) \
                    .reduceByKey(add)

print('The end')

The end


In [ ]:
output = countsRdd.collect()
for(number, count) in output:
  print(number, count)

10 5982
8 5919
4 5905
0 6012
2 5903
6 5996


In [ ]:
linesRdd.count() # total lines of file (not from the sample/amostra)

593805